In [1]:
%reload_ext autoreload
%autoreload 2
import tensorflow as tf
import sys
import math
import numpy as np
sys.path.append('../pytools')
import d2l

d2l.gpu_mem_init()

2025-09-19 15:41:09.223088: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-19 15:41:09.286546: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-19 15:41:10.747886: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Memory growth enabled for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'): True


In [2]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10)
])
x = tf.random.uniform((2, 20, 2))
net(x)

I0000 00:00:1758267671.710976 3532719 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31135 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:00:08.0, compute capability: 7.0
2025-09-19 15:41:11.957574: W external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:237] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 7.0
2025-09-19 15:41:11.957598: W external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:240] Used ptxas at /usr/local/cuda/bin/ptxas
2025-09-19 15:41:11.957644: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:188] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-09-19 15:41:11.959743: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:188] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-09-19 15:41:11.962218: W tensorfl

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.06374773, -0.05352915,  0.24876606, -0.37581503,  0.02651278,
         0.69608605, -0.52900094,  0.12065746,  0.19304925,  0.01788986],
       [-0.14371297, -0.02770487,  0.13521667, -0.3029896 , -0.2068989 ,
         0.48878407, -0.17565036,  0.32239825,  0.4242543 , -0.19951338]],
      dtype=float32)>

In [3]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.hidden = tf.keras.layers.Dense(256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(10)
    def call(self, x):
        return self.out(self.hidden(x))

In [4]:
net = MLP()
x = tf.random.uniform((2, 16))
net(x)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 1.01327404e-01, -2.08832428e-01,  1.81563497e-01,
        -2.29557827e-01, -1.78273141e-01, -3.11898619e-01,
         2.02904701e-01,  3.08429971e-02,  4.09111381e-05,
         3.61326814e-01],
       [-1.76479578e-01, -3.00132453e-01,  2.56714702e-01,
        -2.30226427e-01, -3.88154909e-02, -3.57635140e-01,
         2.33083576e-01,  5.71233407e-02,  1.01431563e-01,
         2.02518225e-01]], dtype=float32)>

In [5]:
class MySequential(tf.keras.Model):
    def __init__(self, *args):
        super().__init__()
        self.models = []
        for blk in args:
            self.models.append(blk)
    def call(self, x):
        for blk in self.models:
            x = blk(x)
        return x

In [6]:
net = MySequential(
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10)
)
net(x)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.09115406, -0.09465532, -0.07273813,  0.03880987,  0.3292024 ,
        -0.12570827,  0.07184786, -0.05869579, -0.09463314,  0.29048485],
       [ 0.01789289,  0.17741422,  0.14595613, -0.07918834,  0.37817797,
        -0.08254399,  0.06946908, -0.06000722, -0.10639155,  0.17654687]],
      dtype=float32)>

In [7]:
def cond(x,y):
    return tf.reduce_sum(tf.abs(x)) > 1
def body(x,y):
    out = y / 2
    return out,out
class FixedHiddenMlp(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.random_weight = tf.random.uniform((20,20))
        self.dense = tf.keras.layers.Dense(20, activation='relu')
    def call(self, x):
        x = self.flatten(x)
        x = tf.nn.relu(tf.matmul(x, self.random_weight) + 1)
        x = self.dense(x)
        #原书中这个地方直接用python表达式，直接调用FixedHiddenMlp可以跑成功；但是在下文中作为tf.keras.Sequential的某一层
        #运行会失败，改成tf.while_loop能运行成功
        out = tf.while_loop(cond, body, [x,x])
        return tf.reduce_sum(out)

In [8]:
net = FixedHiddenMlp()
x = tf.random.uniform((2,20))
net(x)

<tf.Tensor: shape=(), dtype=float32, numpy=1.624274492263794>

In [9]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64, activation='relu'))
        self.net.add(tf.keras.layers.Dense(32, activation='relu'))
        self.dense = tf.keras.layers.Dense(16, activation='relu')
    def call(self, x):
        return self.dense(self.net(x))

In [10]:
mixnet = tf.keras.Sequential()
mixnet.add(NestMLP())
mixnet.add(tf.keras.layers.Dense(20))
mixnet.add(FixedHiddenMlp())

In [11]:
x = tf.random.uniform((2, 20))
y = mixnet(x)
print(x)
print(y)

tf.Tensor(
[[0.270679   0.69886506 0.23550093 0.30138934 0.5368024  0.78038657
  0.9007256  0.18428016 0.764475   0.9831492  0.5891861  0.4123236
  0.73847747 0.7795919  0.42276096 0.97720075 0.87284446 0.5005188
  0.3752228  0.43402815]
 [0.74154425 0.45270002 0.040434   0.0415678  0.32297945 0.9199375
  0.34642684 0.59795105 0.813846   0.4638996  0.40777338 0.944922
  0.3030237  0.8540398  0.65797293 0.5134666  0.5534369  0.58092976
  0.4765967  0.23661697]], shape=(2, 20), dtype=float32)
tf.Tensor(1.5981015, shape=(), dtype=float32)


In [12]:
#print(mixnet.summary())

In [13]:
mixnet.save_weights('mix.weights.h5')

In [16]:
mixnet0 = tf.keras.Sequential()
mixnet0.add(NestMLP())
mixnet0.add(tf.keras.layers.Dense(20))
mixnet0.add(FixedHiddenMlp())
mixnet0(x)
mixnet0.load_weights('mix.weights.h5')
print(x)
print(mixnet0(x))

tf.Tensor(
[[0.270679   0.69886506 0.23550093 0.30138934 0.5368024  0.78038657
  0.9007256  0.18428016 0.764475   0.9831492  0.5891861  0.4123236
  0.73847747 0.7795919  0.42276096 0.97720075 0.87284446 0.5005188
  0.3752228  0.43402815]
 [0.74154425 0.45270002 0.040434   0.0415678  0.32297945 0.9199375
  0.34642684 0.59795105 0.813846   0.4638996  0.40777338 0.944922
  0.3030237  0.8540398  0.65797293 0.5134666  0.5534369  0.58092976
  0.4765967  0.23661697]], shape=(2, 20), dtype=float32)
tf.Tensor(1.4902631, shape=(), dtype=float32)
